based on these:
https://medium.com/geekculture/fine-tuning-bert-with-5-lines-of-code-78803a51c14b
https://huggingface.co/docs/transformers/training

https://discuss.huggingface.co/t/i-have-trained-my-classifier-now-how-do-i-do-predictions/3625

information about tge ct-bert is here:
https://github.com/digitalepidemiologylab/covid-twitter-bert
https://huggingface.co/digitalepidemiologylab/covid-twitter-bert
finetuning seems complicated.

conventional bert models could do the trick, the results may not be that good tho. Uncomment the right cells to use the intended model.

In [1]:
import pandas as pd
import transformers

In [2]:
d = pd.read_excel('tweets_labeled.xlsx')
tweets = d['full_text'][:600]
labels = d['Label (critical: 1/neutral: 0)'][:600]
data = pd.DataFrame({'tweets': tweets, 'labels': labels})

In [3]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data,test_size=0.2,random_state=1)

In [4]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [5]:
# tokenizer = transformers.AutoTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert")

In [5]:
# tokenize function
def tokenize_text(data):
    encoded = tokenizer(data, padding=True, truncation=True, return_tensors='np')
    return encoded.input_ids

In [6]:
train_data = tokenize_text(list(data_train['tweets']))
test_data = tokenize_text(list(data_test['tweets']))

In [7]:
""" for i in range(len(train_data['attention_mask'])):
    for j in range(len(train_data['attention_mask'][i])):
        if train_data['attention_mask'][i][j] == 0:
            if train_data['input_ids'][i][j] != 0:
                print(train_data['input_ids'][i][j]) """

" for i in range(len(train_data['attention_mask'])):\n    for j in range(len(train_data['attention_mask'][i])):\n        if train_data['attention_mask'][i][j] == 0:\n            if train_data['input_ids'][i][j] != 0:\n                print(train_data['input_ids'][i][j]) "

In [8]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [9]:
# model = transformers.AutoModel.from_pretrained("digitalepidemiologylab/covid-twitter-bert")

In [10]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import tensorflow as tf
import numpy as np

In [11]:
train_labels = np.array(data_train['labels']).astype(np.int64)
test_labels = np.array(data_test['labels']).astype(np.int64)

In [12]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=tf.metrics.SparseCategoricalAccuracy())
model.fit(
    train_data,
    train_labels,
    validation_data=(test_data, test_labels),
    epochs=10,
    batch_size=32
)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
15/15 [==============================] - 301s 19s/step - loss: 0.5701 - sparse_categorical_accuracy: 0.7063 - val_loss: 0.6619 - val_sparse_categorical_accuracy: 0.7000
Epoch 2/10
15/15 [==============================] - 311s 21s/step - loss: 0.46

AttributeError: 'BertModel' object has no attribute 'predict'

In [13]:
from transformers import TextClassificationPipeline

In [14]:
from transformers import pipeline
import json

In [16]:
""" pipe = pipeline(task='fill-mask', model=model, tokenizer=tokenizer)
out = pipe(f"In places with a lot of people, it's a good idea to wear a {pipe.tokenizer.mask_token}")
print(json.dumps(out, indent=4)) """

' pipe = pipeline(task=\'fill-mask\', model=model, tokenizer=tokenizer)\nout = pipe(f"In places with a lot of people, it\'s a good idea to wear a {pipe.tokenizer.mask_token}")\nprint(json.dumps(out, indent=4)) '

In [17]:
pipe = TextClassificationPipeline(model = model, tokenizer = tokenizer, return_all_scores=True)

In [19]:
pipe("RT @EricWishart: Because of #COVID, the mask shop is closed. #HongKong #China https://t.co/FYcJX1pzNK")

[[{'label': 'LABEL_0', 'score': 0.011910678818821907},
  {'label': 'LABEL_1', 'score': 0.988089382648468}]]

In [21]:
test_tweets = d["full_text"][600:615]
for t in test_tweets:
    print(t)
    print(pipe(t))

Total Covid Death Distribution by County  For ID   2022-03-04:  Latest Covid Insights by Our Analytics Team using USAFacts #datavisualization #datascience #analytics #healthtech #data #covid19 #publichealth #covid #globalhealth #RStats https://t.co/pFJkoMSCwH
[[{'label': 'LABEL_0', 'score': 0.9981666803359985}, {'label': 'LABEL_1', 'score': 0.0018332962645217776}]]
RT @healthcareontim: Heart disease refers to several types of conditions that affect the heart. https://t.co/FdgGiB8m6E
#disease #health #c…
[[{'label': 'LABEL_0', 'score': 0.9947262406349182}, {'label': 'LABEL_1', 'score': 0.005273723974823952}]]
Heart disease refers to several types of conditions that affect the heart. https://t.co/FdgGiB8m6E
#disease #health #coronavirus #virus #corona #covid19 #covid_19 #medicine #love #illness #pandemic #covid #quarantine #stayhome #cancer #life #china #prevention #wellness #healthy
[[{'label': 'LABEL_0', 'score': 0.9947708249092102}, {'label': 'LABEL_1', 'score': 0.00522918626666069}]]

In [25]:
pipe("Did #VladimirPutin scare #Covid off or what ? It’s literally not on the news at all after dominating our existence over the last few years 🤔 ou’ve gotta be a bit of an ostrich to trust our government these days")

[[{'label': 'LABEL_0', 'score': 0.015024186111986637},
  {'label': 'LABEL_1', 'score': 0.9849758744239807}]]

In [27]:
# save model
model.save_pretrained('models/tf_model')

In [28]:
tokenizer.save_pretrained('models/tf_tokenizer')

('models/tf_tokenizer\\tokenizer_config.json',
 'models/tf_tokenizer\\special_tokens_map.json',
 'models/tf_tokenizer\\vocab.txt',
 'models/tf_tokenizer\\added_tokens.json',
 'models/tf_tokenizer\\tokenizer.json')

In [30]:
pipe("we should stop wearing masks")

[[{'label': 'LABEL_0', 'score': 0.4587249755859375},
  {'label': 'LABEL_1', 'score': 0.5412750244140625}]]